In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from data.data_wizard import data_wizard
from models.ExpNorm import ExpNorm2d, ExpNorm1d
import math
import random
import numpy as np

In [2]:
train_loader_batch, _, _ = data_wizard(
    name='cifar100',
    batch_size=200,
    val_par=0.1
)
train_loader_ol, _, _ = data_wizard(
    name='cifar100',
    batch_size=1,
    val_par=0.1
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
EPOCHS_OL = 1
EPOCHS_B = EPOCHS_OL*1

In [4]:
bnorm = nn.BatchNorm2d(3, affine=False).to('cuda:0')
expnorm = ExpNorm2d(3, affine=False, eps=0.5).to('cuda:0')

In [5]:
bnorm.train()
for ep in range(EPOCHS_B):
    for img, _ in train_loader_batch:
        out = bnorm(img.to('cuda:0'))
bnorm.eval()
print("With BatchNorm, mean:{}, std:{}".format(bnorm.running_mean, bnorm.running_var))

With BatchNorm, mean:tensor([0.4962, 0.4683, 0.4196], device='cuda:0'), std:tensor([0.0669, 0.0603, 0.0691], device='cuda:0')


In [6]:
expnorm.train()
for ep in range(EPOCHS_OL):
    for img, _ in train_loader_batch:
        out = expnorm(img.to('cuda:0'))
expnorm.eval()
print("With ExpNorm, mean:{}, std:{}".format(expnorm.running_mean, expnorm.running_var))

With ExpNorm, mean:tensor([0.4978, 0.4679, 0.4193], device='cuda:0'), std:tensor([0.0667, 0.0601, 0.0688], device='cuda:0')


In [7]:
# Numerical study
N_STEP = 1000000
exp_avg = 0
for _ in range(1000000):
    exp_avg = exp_avg*0.9 + 0.1*random.random()
print(f"Exp avg:{exp_avg}")
# Normal avg
sum_normal = 0
for _ in range(1000000):
    sum_normal += random.random()
print(f"Noraml avg:{sum_normal/1000000}")

Exp avg:0.4865380189675993
Noraml avg:0.5000315170590531


In [8]:
expnorm1d = ExpNorm1d(3).train()
for _ in range(10000):
    expnorm1d(torch.rand([1,3]))
expnorm1d.eval()
print("With ExpNorm, mean:{}, std:{}".format(expnorm1d.running_mean, expnorm1d.running_var))

With ExpNorm, mean:tensor([ 0.0121, -0.0083,  0.0176]), std:tensor([1.0016, 1.0090, 0.9884])
